# Data Preparation

1. [Import Libraries and Load Raw Data](#Import-Libraries-and-Load-Raw-Data)
2. [Date and Time Processing](#Date-and-Time-Processing)
3. [Remove Missing Data](#Remove-Missing-Data)
4. [Date Range Filtering](#Date-Range-Filtering)
5. [Add Stock Data](#Add-Stock-Data)
6. [Drop Duplicates](#Drop-Duplicates)
7. [Filter out Uninformative Headlines](#Filter-out-Uninformative-Headlines)
8. [Remove Similar Headlines](#Remove-Similar-Headlines)
9. [Filter for Valid Trading Sessions after Headline](#Filter-for-Valid-Trading-Sessions-after-Headline)
10. [Calculate Trading Returns](#Calculate-Trading-Returns)
11. [Stratified Sampling](#Stratified-Sampling)

## Import Libraries and Load Raw Data

In [2]:
import json
import os
import pandas as pd
import requests
import io
from datetime import datetime, timedelta
import time
import yfinance as yf
#from pandas_datareader import data
from typing import Optional
import re
import string
from textdistance import damerau_levenshtein
from itertools import combinations
from tqdm import tqdm

input_path = "source/analyst_ratings_processed.csv"  

df = pd.read_csv(input_path)
df = df[['title', 'date', 'stock']]
df.head()

,title,date,stock
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


## Date and Time Processing
Restructure the datetime information, splitting into separate date, time and offset columns for easier processing below.

In [2]:
df.rename(columns={'date': 'datetime'}, inplace=True)

df['date'] = df['datetime']
df['time'] = df['date'].str.split(' ').str[1]
df['date'] = df['date'].str.split(' ').str[0]
df['offset'] = df['time'].str.split('-').str[1]
df['time'] = df['time'].str.split('-').str[0]
df.head()

,title,datetime,stock,date,time,offset
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,2020-06-05,10:30:00,04:00
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,2020-06-03,10:45:00,04:00
2,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,2020-05-26,04:30:00,04:00
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,2020-05-22,12:45:00,04:00
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,2020-05-22,11:38:00,04:00


## Remove Missing Data
Filter out rows with missing datetime values. Datetime objects with midnight timestamps (00:00:00) are also removed as this implies that no time data was available and the default value was used. 

In [3]:
print(f"Values with no time to be removed: {len(df[df['time'] == '00:00:00'])}")
print(f"Null datetime values to be removed: {len(df[df['datetime'].isna()])}")

df = df[df['time'] != '00:00:00']
df = df[~df['datetime'].isna()]

df = df[['title', 'datetime', 'stock', 'date', 'time', 'offset']]
df.to_csv('output2/filtered_data.csv', index=False)

Values with no time to be removed: 640
Null datetime values to be removed: 0


## Date Range Filtering
Remove all headlines outside the research period.

In [4]:
def filter_stock_data(input_path, output_path):
    try:
        df = pd.read_csv(input_path)
        df['date'] = pd.to_datetime(df['date'])
        
        # Create datetime objects for comparison
        start_date = pd.to_datetime('2018-10-01')
        end_date = pd.to_datetime('2019-12-31')
        
        # Filter data between the specified dates
        mask = (df['date'] >= start_date) & (df['date'] <= end_date)
        filtered_df = df.loc[mask]
        
        # Save filtered data 
        filtered_df.to_csv(output_path, index=False)
        
        print(f"Original headlines: {len(df)}")
        print(f"Filtered headlines: {len(filtered_df)}")
        
        return filtered_df
        
    except FileNotFoundError:
        print("Error: CSV file not found")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

input_path = "output2/filtered_data.csv"  
output_path = "output2/filtered_data2.csv" 
filtered_data = filter_stock_data(input_path, output_path)

Original headlines: 1398540
Filtered headlines: 191689


## Add Stock Data
This section enriches the dataset by adding: 

- Exchange
- Share code
- Company name

to each row. Afterwards stocks are filtered to only those with share codes 10/11.


In [5]:
def determine_share_code(security_name):
    if pd.isna(security_name):
        return 10
        
    if not isinstance(security_name, str):
        security_name = str(security_name)
        
    unit_indicators = [
        ' UNIT',
        ' UNITS',
        'UNIT ',
        'UNITS ',
        ' AND ',
        '/WRNTS',
        '/WRNT',
        '/RTS',
        'RIGHTS',
        'PAIRED SHARE',
        ' PER SHARE',
        ' - UNIT',
        ' - UNITS',
        'DEPOSITARY SHARE'
    ]
    
    if any(indicator in security_name.upper() for indicator in unit_indicators):
        return 11
    return 10

def download_exchange_data():
    """
    Downloads current ticker data from NASDAQ FTP site
    """
    print("Downloading exchange data...")
    
    urls = {
        'NASDAQ': 'https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt',
        'NYSE/AMEX': 'https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt'
    }
    
    all_data = []
    
    for exchange_name, url in urls.items():
        try:
            print(f"Fetching {exchange_name} data...")
            response = requests.get(url)
            if response.status_code == 200:
                # Read the data using pandas directly
                df = pd.read_csv(io.StringIO(response.text), delimiter='|', comment='#')
                
                # Ensure Symbol column is string type
                if 'Symbol' in df.columns:
                    df['Symbol'] = df['Symbol'].astype(str)
                if 'ACT Symbol' in df.columns:
                    df['ACT Symbol'] = df['ACT Symbol'].astype(str)
                
                # For NASDAQ file
                if 'nasdaqtraded.txt' in url:
                    df = df.rename(columns={
                        'Symbol': 'Symbol',
                        'Security Name': 'Security_Name',
                        'ETF': 'ETF',
                        'Round Lot Size': 'Round_Lot_Size',
                        'Test Issue': 'Test_Issue'
                    })
                    # NASDAQ file needs exchange set explicitly
                    df['Exchange'] = df['Market Category'].apply(lambda x: 'NASDAQ' if x in ['Q', 'G', 'S'] else None)
                
                # For NYSE/AMEX file
                else:
                    df = df.rename(columns={
                        'ACT Symbol': 'Symbol',
                        'Security Name': 'Security_Name',
                        'Exchange': 'Exchange',
                        'ETF': 'ETF',
                        'Round Lot Size': 'Round_Lot_Size',
                        'Test Issue': 'Test_Issue'
                    })
                
                # Add share code based on security name
                df['Share_Code'] = df['Security_Name'].apply(determine_share_code)
                
                print(f"Found {len(df)} records from {exchange_name}")
                print("Sample symbols and share codes:")
                print(df[['Symbol', 'Security_Name', 'Share_Code']].head())
                
                all_data.append(df)
            
            time.sleep(1)  # Be nice to the server
            
        except Exception as e:
            print(f"Error downloading {exchange_name} data: {str(e)}")
            if 'response' in locals():
                print("Response status:", response.status_code)
                print("Response headers:", response.headers)
                print("First 500 chars of response:", response.text[:500])
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        # Clean up the data
        combined_df = combined_df[
            (combined_df['Test_Issue'] == 'N') &  # Remove test issues
            (combined_df['ETF'] == 'N')  # Remove ETFs
        ]
        
        # Map exchange codes to names
        exchange_map = {
            'N': 'NYSE',
            'A': 'AMEX',
            'Q': 'NASDAQ',
            'P': 'NYSE',  # NYSE Arca
            'Z': 'NYSE',  # NYSE BATS
            'V': 'NYSE',  # NYSE IEX
            'NASDAQ': 'NASDAQ',  # Direct NASDAQ listing
            'NYSE': 'NYSE',      # Direct NYSE listing
            'AMEX': 'AMEX'       # Direct AMEX listing
        }
        
        combined_df['Exchange'] = combined_df['Exchange'].map(exchange_map)
        combined_df = combined_df[combined_df['Exchange'].notna()]
        
        print(f"\nTotal valid records after filtering: {len(combined_df)}")
        print("\nExchange distribution:")
        print(combined_df['Exchange'].value_counts())
        print("\nShare code distribution:")
        print(combined_df['Share_Code'].value_counts())
        
        return combined_df
    
    return pd.DataFrame()


def clean_company_names(df):
    terms_to_remove = [
        "Inc", "Common", "Stock", "Equity", "Registry", "Shares", 
        "Limited", "Corp", "Corporation", "Holdings", "Industries", 
        "Depository", "Ordinary", "Class B", "Class A", "Class C"
    ]
    
    # Create a copy to avoid modifying the original DataFrame
    df_clean = df.copy()
    
    # Function to clean each company name
    def clean_name(name):
        if not isinstance(name, str):
            return name
            
        # Remove each term (with word boundaries)
        cleaned_name = name
        for term in terms_to_remove:
            pattern = r'\b' + re.escape(term) + r'\b'
            cleaned_name = re.sub(pattern, '', cleaned_name)
        
        # Remove punctuation
        cleaned_name = cleaned_name.translate(str.maketrans('', '', string.punctuation))
        
        # Remove any double spaces created and trim
        cleaned_name = " ".join(cleaned_name.split())
        
        return cleaned_name.strip()
    
    # Apply the cleaning function
    df_clean['company_name'] = df_clean['company_name'].apply(clean_name)
    
    return df_clean

def process_news_data(input_file, output_file):
    """
    Process news CSV file and filter based on exchange listings
    """
    try:
        # Read the input CSV
        df = pd.read_csv(input_file)
        
        # Ensure stock column is string type
        df['stock'] = df['stock'].astype(str)
        
        # Print unique stocks for debugging
        unique_stocks = df['stock'].nunique()
        print(f"\nUnique stocks in input: {unique_stocks}")
        print("Most common stocks in input:")
        print(df['stock'].value_counts().head())
        
        # Get exchange data
        exchange_df = download_exchange_data()
        
        if exchange_df.empty:
            raise Exception("Could not obtain exchange data")
        
        # Create ticker to info mapping (exchange, share code, and company name)
        # Ensure all symbols are strings and uppercase
        ticker_info_map = dict(zip(
            exchange_df['Symbol'].astype(str).str.strip().str.upper(),
            zip(exchange_df['Exchange'], exchange_df['Share_Code'], exchange_df['Security_Name'])
        ))
        
        print("\nSample of ticker mapping (Symbol: Exchange, Share Code, Company):")
        sample_tickers = list(ticker_info_map.items())[:5]
        for ticker, (exchange, share_code, company_name) in sample_tickers:
            print(f"{ticker}: {exchange}, {share_code}, {company_name}")
        
        # Process news data
        df['stock_upper'] = df['stock'].astype(str).str.strip().str.upper()
        df['exchange'] = None
        df['share_code'] = None
        df['company_name'] = None
               
        # Apply exchange, share code, and company name information
        for idx, row in df.iterrows():
            if row['stock_upper'] in ticker_info_map:
                exchange, share_code, company_name = ticker_info_map[row['stock_upper']]
                df.at[idx, 'exchange'] = exchange
                df.at[idx, 'share_code'] = share_code
                df.at[idx, 'company_name'] = company_name
        
        # Remove temporary column
        df = df.drop('stock_upper', axis=1)
        
        # Filter and sort
        df_filtered = df[df['exchange'].notna()].copy()
        
        print(f"\nFiltering results:")
        print(f"Original rows: {len(df)}")
        print(f"Filtered rows: {len(df_filtered)}")
        print(f"\nMatches by exchange:")
        print(df_filtered['exchange'].value_counts())
        print(f"\nMatches by share code:")
        print(df_filtered['share_code'].value_counts())

        # Clean company names
        df_cleaned = clean_company_names(df_filtered)
        
        # Save results
        df_cleaned.to_csv(output_file, index=False)
        
        print(f"\nResults saved to: {output_file}")
        if not df_cleaned.empty:
            print("\nSample of processed data:")
            print(df_cleaned[['title', 'date', 'stock', 'exchange', 'share_code', 'company_name']].head())
            
            # Print some statistics
            print("\nTop 10 stocks in filtered data:")
            print(df_cleaned['stock'].value_counts().head(10))
        else:
            print("\nNo matches found in the data")
        
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        raise



input_file = "output2/filtered_data2.csv"  
output_file = "output2/filtered_data3.csv"   
process_news_data(input_file, output_file)



Unique stocks in input: 4141
Most common stocks in input:
stock
PCG      1034
NFLX      978
GOOGL     941
EWU       907
DIA       838
Name: count, dtype: int64
Fetching NASDAQ data...
Found 11257 records from NASDAQ
Sample symbols and share codes:
  Symbol                                   Security_Name  Share_Code
0      A         Agilent Technologies, Inc. Common Stock          10
1     AA                 Alcoa Corporation Common Stock           10
2    AAA  Alternative Access First Priority CLO Bond ETF          10
3   AAAU          Goldman Sachs Physical Gold ETF Shares          10
4  AACBU              Artius II Acquisition Inc. - Units          11
Fetching NYSE/AMEX data...
Found 6437 records from NYSE/AMEX
Sample symbols and share codes:
  Symbol                                      Security_Name  Share_Code
0      A            Agilent Technologies, Inc. Common Stock          10
1     AA                    Alcoa Corporation Common Stock           10
2    AAA     Alternative Acc

## Drop Duplicates
Remove duplicates based on title, date, and stock to prevent bias from repeated headlines.

In [6]:
def remove_duplicates(input_path, output_path):

    df = pd.read_csv(input_path)  
    initial_count = len(df)
    
    df_deduped = df.drop_duplicates(subset=['title', 'date', 'stock'], keep='first')
    final_count = len(df_deduped)
    
    df_deduped.to_csv(output_path, index=False)
    
    return {
        'initial_count': initial_count,
        'final_count': final_count,
        'duplicates_removed': initial_count - final_count
    }


input_file = "output2/filtered_data3.csv"
output_file = "output2/filtered_data4.csv"

results = remove_duplicates(input_file, output_file)
print(f"Initial row count: {results['initial_count']}")
print(f"Updated row count: {results['final_count']}")
print(f"Duplicates removed: {results['duplicates_removed']}")

Initial row count: 129759
Updated row count: 129699
Duplicates removed: 60


## Drop Low Relevance Headlines
For our purposes, a headline is considered low relevance if it doesn't either contain the name of the stock ticker or the company name in the headline. As part of this, stock tickers with only 1 letter will be dropped.

In [7]:
def filter_titles_by_relevance(input_path, output_path):
    df = pd.read_csv(input_path)
    
    # Keep track of original row count
    original_count = len(df)

    # Drop stock tickers with only a single character
    mask = df['stock'].astype(str).str.len() > 1
    df = df[mask]
    
    def contains_relevant_words(row):
        if pd.isna(row['title']):
            return False
            
        title = str(row['title']).lower()
        
        # Get words from stock and company_name
        stock_words = []
        if not pd.isna(row['stock']):
            stock_words = [word.lower() for word in re.split(r'\s+', str(row['stock'])) if word]
            
        company_words = []
        if not pd.isna(row['company_name']):
            company_words = [word.lower() for word in re.split(r'\s+', str(row['company_name'])) if len(word) > 1]
            
        all_words = stock_words + company_words
        
        # Check if any word from stock or company_name appears in the title
        return any(word in title for word in all_words if word)
    
    # Apply the filter
    filtered_df = df[df.apply(contains_relevant_words, axis=1)]
    
    # Calculate statistics
    filtered_count = len(filtered_df)
    dropped_count = original_count - filtered_count
    
    stats = {
        'original_rows': original_count,
        'filtered_rows': filtered_count,
        'dropped_rows': dropped_count,
        'percentage_kept': round(filtered_count / original_count * 100, 2) if original_count > 0 else 0
    }
    
    filtered_df.to_csv(output_path, index=False)
        
    return filtered_df, stats

input_path = "output2/filtered_data4.csv"
output_path = "output2/filtered_data5.csv"

filtered_data, stats = filter_titles_by_relevance(input_path, output_path)

print(f"Original rows: {stats['original_rows']}")
print(f"Dropped rows: {stats['dropped_rows']}")
print(f"Reminaing rows: {stats['filtered_rows']}")
print(f"Percentage kept: {stats['percentage_kept']}%")

filtered_data.head()

Original rows: 129699
Dropped rows: 65652
Reminaing rows: 64047
Percentage kept: 49.38%


,title,datetime,stock,date,time,offset,exchange,share_code,company_name
103,Atlantic American Q3 EPS $(0.07) Down From $0....,2019-11-12 14:11:00-05:00,AAME,2019-11-12,14:11:00,05:00,NASDAQ,10,Atlantic American
107,Atlantic American Q1 EPS $0.19 Up From $(0.25)...,2019-05-13 10:04:00-04:00,AAME,2019-05-13,10:04:00,04:00,NASDAQ,10,Atlantic American
108,Atlantic American Q4 EPS $0.01 Down From $0.12...,2019-04-01 14:04:00-04:00,AAME,2019-04-01,14:04:00,04:00,NASDAQ,10,Atlantic American
109,Atlantic American Q3 EPS $0.04 Up From $0.03 Y...,2018-11-13 12:14:00-05:00,AAME,2018-11-13,12:14:00,05:00,NASDAQ,10,Atlantic American
111,Applied Optoelectronics to Participate in Raym...,2019-12-03 16:11:00-05:00,AAOI,2019-12-03,16:11:00,05:00,NASDAQ,10,Applied Optoelectronics


## Drop Uninformative Headlines
Filter out uninformative headlines (like "10 Stocks to Watch" or "Stocks That Hit 52-Week Highs") that don't provide any meaningful sentiment for a stock.

In [8]:
def is_uninformative_headline(headline):
    patterns = [
        r'^\d+\s+(Stocks|Companies|Biggest|Largest)',  # "74 Biggest Movers", "10 Stocks"
        r'^\d+\s+Stocks (Moving|To Watch)',  # "Stocks Moving", "Stocks To Watch"
        r'Stocks That Hit',  # "Stocks That Hit 52-Week Highs"
        r'(Top|Biggest) (Upgrades|Downgrades)',  # Benzinga's Top Upgrades/Downgrades
        r'Earnings Scheduled For',  # Earnings calendar entries
        r'Pre-Market Session$',  # Pre-market movement summaries
        r'Price Target Changes',  # Price target updates without specifics
        r'^\d+.*Moving In',  # "7 Technology Stocks Moving In"
        r'52-Week (Highs|Lows)',  # 52-week high/low lists
    ]
    
    # Check if headline matches any of the patterns
    for pattern in patterns:
        if re.search(pattern, headline, re.IGNORECASE):
            # Additional check: if the headline contains ":" followed by a company name,
            # it might still be informative
            if ":" in headline:
                return False
            return True
            
    return False

def filter_headlines(df, headline_column='title'):
    # Create mask of informative headlines
    mask = ~df[headline_column].apply(is_uninformative_headline)
    
    # Split into informative and uninformative headlines
    informative_headlines = df[mask]
    removed_headlines = df[~mask]
    
    return informative_headlines, removed_headlines

# Test the function with sample headlines
test_headlines = [
    "74 Biggest Movers From Yesterday",
    "8 Stocks To Watch For December 17, 2019",
    "Mid-Morning Market Update: Markets Edge Higher; Jabil Beats Q1 Expectations",
    "30 Stocks Moving in Tuesday's Pre-Market Session",
    "9 Energy Stocks Moving In Wednesday's Pre-Market Session"
]

print("Sample classifications:")
print()
for headline in test_headlines:
    result = "Uninformative" if is_uninformative_headline(headline) else "Informative"
    print(f"\nHeadline: {headline}")
    print(f"Classification: {result}")
    

input_path = "output2/filtered_data5.csv"
output_path = "output2/filtered_data6.csv"    

df = pd.read_csv(input_path)
informative_df, removed_df = filter_headlines(df)
print()
print(f"Kept {len(informative_df)} headlines, removed {len(removed_df)} headlines")
print()
print(f"First 20 kept headlines: ")
print(informative_df['title'].head(20))
print()
print(f"First 20 removed headlines: ")
print(removed_df['title'].head(20))

informative_df.to_csv(output_path)
removed_df.to_csv('output2/removed_irrelevant_headlines.csv', index=False)

Sample classifications:


Headline: 74 Biggest Movers From Yesterday
Classification: Uninformative

Headline: 8 Stocks To Watch For December 17, 2019
Classification: Uninformative

Headline: Mid-Morning Market Update: Markets Edge Higher; Jabil Beats Q1 Expectations
Classification: Informative

Headline: 30 Stocks Moving in Tuesday's Pre-Market Session
Classification: Uninformative

Headline: 9 Energy Stocks Moving In Wednesday's Pre-Market Session
Classification: Uninformative

Kept 63544 headlines, removed 503 headlines

First 20 kept headlines: 
0     Atlantic American Q3 EPS $(0.07) Down From $0....
1     Atlantic American Q1 EPS $0.19 Up From $(0.25)...
2     Atlantic American Q4 EPS $0.01 Down From $0.12...
3     Atlantic American Q3 EPS $0.04 Up From $0.03 Y...
4     Applied Optoelectronics to Participate in Raym...
5     Applied Optoelectronics Amended S-3 Shows Regi...
6     Piper Jaffray Maintains Neutral on Applied Opt...
7     Applied Optoelectronics Sees Q4 Adj. EPS $(0.2.

## Remove Similar Headlines
Use Damerau-Levenshtein similarity to remove nearly identical headlines about the same stock on the same day. 

In [9]:
def remove_similar_headlines(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    
    groups = df.groupby(['stock', 'date'])
    rows_to_drop = set()
    similar_pairs = []
    
    # Get total number of comparisons for progress bar
    total_comparisons = sum(len(list(combinations(group.index, 2))) 
                          for _, group in groups if len(group) > 1)
    
    # Create progress bar
    pbar = tqdm(total=total_comparisons, desc="Comparing headlines")
    current_comparisons = 0
    
    for (stock, date), group in groups:
        if len(group) <= 1:
            continue
            
        for idx1, idx2 in combinations(group.index, 2):
            headline1 = group.loc[idx1, 'title']
            headline2 = group.loc[idx2, 'title']
            
            similarity = damerau_levenshtein.normalized_similarity(headline1, headline2)
            
            if similarity > 0.6:
                rows_to_drop.add(idx2)
                similar_pairs.append({
                    'stock': stock,
                    'date': date,
                    'headline1': headline1,
                    'headline2': headline2,
                    'similarity': round(similarity, 3)
                })
            
            # Update progress bar
            pbar.update(1)
    
    pbar.close()
    
    print(f"\nTotal headlines removed: {len(rows_to_drop)}")
    print("\nExample similar headlines removed:")
    for pair in similar_pairs[:5]:
        print(f"\nStock: {pair['stock']} - Date: {pair['date']}")
        print(f"Similarity: {pair['similarity']}")
        print(f"1: {pair['headline1']}")
        print(f"2: {pair['headline2']}")
    
    return df.drop(index=rows_to_drop).reset_index(drop=True)

input_path = "output2/filtered_data6.csv"
output_path = "output2/filtered_data7.csv" 

df = pd.read_csv(input_path)
print(f"Loaded {len(df)} rows")

deduped_df = remove_similar_headlines(df)

deduped_df.to_csv(output_path, index=False)
print(f"Saved {len(deduped_df)} rows to {output_path}")

Loaded 63544 rows


Comparing headlines: 100%|██████████████████████████████████████████████████████████████| 47500/47500 [04:53<00:00, 161.57it/s]



Total headlines removed: 2451

Example similar headlines removed:

Stock: AAOI - Date: 2019-11-06 00:00:00
Similarity: 0.61
1: Applied Optoelectronics Sees Q4 Adj. EPS $(0.23)-$(0.21) vs $(0.18) Est., Sales $46M-$49M vs $51.84M Est.
2: Applied Optoelectronics Q3 Adj. EPS $(0.15) Beats $(0.24) Estimate, Sales $46.1M Miss $47.75M Estimate

Stock: AAON - Date: 2019-10-31 00:00:00
Similarity: 0.813
1: AAON Q3 EPS $0.26 Down From $0.27 YoY, Sales $113.5M Up From $112.937M YoY
2: AAON Earlier Reported Q3 EPS $0.26 Down From $0.27 YoY, Sales $113.5M Up From $112.937M YoY

Stock: AAP - Date: 2018-11-14 00:00:00
Similarity: 0.744
1: Deutsche Bank Maintains Buy on Advance Auto Parts, Raises Price Target to $205
2: Citigroup Maintains Neutral on Advance Auto Parts, Raises Price Target to $185

Stock: AAP - Date: 2018-11-14 00:00:00
Similarity: 0.833
1: Deutsche Bank Maintains Buy on Advance Auto Parts, Raises Price Target to $205
2: UBS Maintains Buy on Advance Auto Parts, Raises Price Target to

## Filter for Valid Trading Sessions after Headline
Filter headlines on whether they can be associated with a valid subsequent trading session.

In [10]:
def filter_headlines_by_trading_session(df):
    """
    Filter headlines based on trading session rules:
    - Before 6 AM: Same day trading (open to close)
    - 6 AM to 4 PM: Same day close to next day close
    - After 4 PM: Next day open to next day close
    """
    
    # Convert date column to datetime if it's not already
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    #df['date'] = df['date'].dt.tz_convert('America/New_York')
    #df['date'] = df['date'].dt.tz_localize('America/New_York') 
    
    # Create a copy to avoid modifying the original DataFrame
    df_filtered = df.copy()
    
    # Add helper columns for filtering
    df_filtered['day_of_week'] = df_filtered['date'].dt.dayofweek  # Monday = 0, Sunday = 6
    df_filtered['hour'] = df_filtered['time'].str.split(':').str[0]
    df_filtered['hour'] = df_filtered['hour'].astype(int)
    df_filtered['minute'] = df_filtered['time'].str.split(':').str[1]
    df_filtered['minute'] = df_filtered['minute'].astype(int)
    
    # Filter conditions
    def has_valid_trading_session(row):
        # Skip weekends
        if row['day_of_week'] in [5, 6]:
            return False
            
        # Before 9:30 AM - needs same day trading
        if row['hour'] < 9 or (row['hour'] == 9 and row['minute'] < 30):
            return True  
            
        # Between 9:30 AM and 4 PM - needs next trading day
        elif (row['hour'] == 9 and row['minute'] >= 30) or (row['hour'] >= 10 and row['hour'] < 16):
            next_date = row['date'] + timedelta(days=1)
            while next_date.dayofweek in [5, 6]:  # Skip weekends
                next_date += timedelta(days=1)
            return True  
            
        # After 4 PM - needs next trading day
        else:  # hour >= 16
            next_date = row['date'] + timedelta(days=1)
            while next_date.dayofweek in [5, 6]:  # Skip weekends
                next_date += timedelta(days=1)
            return True  
    
    # Apply filtering
    mask = df_filtered.apply(has_valid_trading_session, axis=1)
    df_filtered = df_filtered[mask]
    
    # Drop helper columns
    df_filtered = df_filtered.drop(['day_of_week', 'hour'], axis=1)
    
    return df_filtered

input_path = 'output2/filtered_data7.csv'
output_path = 'output2/filtered_data8.csv'

df = pd.read_csv(input_path)

filtered_df = filter_headlines_by_trading_session(df)
print(f"Original headlines: {len(df)}")
print(f"Filtered headlines: {len(filtered_df)}")
filtered_df.to_csv(output_path, index=False)

Original headlines: 61093
Filtered headlines: 60421


## Calculate Trading Returns
Retrieve stock price data then calculate returns for the trading session following each headline:

Pre-market headlines: same-day open to close
During market: same-day close to next-day close
After-market: next-day open to close

In [11]:
from tqdm.auto import tqdm

def get_trading_returns(headlines_df, cache_dir='price_cache', batch_size=50, start_date=None, end_date=None):
    
    os.makedirs(cache_dir, exist_ok=True)
    headlines_df['date'] = pd.to_datetime(headlines_df['date'])
    
    if not start_date:
        start_date = headlines_df['date'].min().strftime('%Y-%m-%d')
    if not end_date:
        end_date = (headlines_df['date'].max() + timedelta(days=5)).strftime('%Y-%m-%d')
    
    result_df = headlines_df.copy()    
    unique_stocks = list(headlines_df['stock'].unique())
    
    # Load existing cache manifest if it exists
    manifest_path = os.path.join(cache_dir, 'manifest.json')
    if os.path.exists(manifest_path):
        with open(manifest_path, 'r') as f:
            processed_stocks = set(json.load(f))
    else:
        processed_stocks = set()
    
    # Get stocks that still need processing
    stocks_to_process = [s for s in unique_stocks if s not in processed_stocks]
    
    price_cache = {}
    
    # Load existing cache files
    for stock in processed_stocks:
        cache_file = os.path.join(cache_dir, f"{stock}.csv")
        if os.path.exists(cache_file):
            try:
                price_cache[stock] = pd.read_csv(cache_file)
                price_cache[stock]['Date'] = pd.to_datetime(price_cache[stock]['Date'])
            except Exception as e:
                print(f"Error loading cache for {stock}: {str(e)}")
                if stock in processed_stocks:
                    processed_stocks.remove(stock)
    
    # Fetch price data for remaining stocks with progress bar
    print(f"{len(stocks_to_process)} stocks not found in cache, attempting to fetch...")
    
    for i in tqdm(range(0, len(stocks_to_process), batch_size)):
        batch = stocks_to_process[i:i+batch_size]
        
        for stock in batch:
            try:
                # Add delay between requests
                time.sleep(0.5)  # 500ms delay
                
                # Get data from yfinance
                ticker = yf.Ticker(stock)
                hist = ticker.history(start=start_date, end=end_date)
                
                if not hist.empty:
                    # Reset index to make date a column
                    hist = hist.reset_index()
                    hist['Date'] = pd.to_datetime(hist['Date'])
                    price_data = hist[['Date', 'Open', 'Close']].copy()
                    
                    # Save to cache file
                    cache_file = os.path.join(cache_dir, f"{stock}.csv")
                    price_data.to_csv(cache_file, index=False)
                    
                    # Store in memory cache
                    price_cache[stock] = price_data
                    processed_stocks.add(stock)
                    
            except Exception as e:
                print(f"Error fetching data for {stock}: {str(e)}")
                continue
        
        # Save progress after each batch
        with open(manifest_path, 'w') as f:
            json.dump(list(processed_stocks), f)
    
    def get_trading_prices(row):
        headline_date = pd.to_datetime(row['datetime'])
        headline_hour = headline_date.hour
        headline_minute = headline_date.minute
        headline_day = headline_date.date()
        stock = row['stock']
        
        def get_next_trading_day(current_date):
            next_date = current_date + timedelta(days=1)
            while next_date.weekday() in [5, 6]:  # Skip weekends
                next_date += timedelta(days=1)
            return next_date
        
        if stock not in price_cache:
            return {
                'entry_price': None,
                'exit_price': None
            }
        
        stock_prices = price_cache[stock]
        
        # Before 9:30 AM: Same day open to same day close
        if headline_hour < 9 or (headline_hour == 9 and headline_minute < 30):
            same_day_prices = stock_prices[stock_prices['Date'].dt.date == headline_day]
            if len(same_day_prices) > 0:
                return {
                    'entry_price': same_day_prices.iloc[0]['Open'],
                    'exit_price': same_day_prices.iloc[0]['Close']
                }
        
        # 9:30 AM to 4 PM: Same day close to next trading day close
        elif (headline_hour == 9 and headline_minute >= 30) or (headline_hour >= 10 and headline_hour < 16):
            same_day_prices = stock_prices[stock_prices['Date'].dt.date == headline_day]
            next_trading_day = get_next_trading_day(headline_day)
            next_day_prices = stock_prices[stock_prices['Date'].dt.date == next_trading_day]
            
            if len(same_day_prices) > 0 and len(next_day_prices) > 0:
                return {
                    'entry_price': same_day_prices.iloc[0]['Close'],
                    'exit_price': next_day_prices.iloc[0]['Close']
                }
        
        # After 4 PM: Next trading day open to close
        else:  # headline_hour >= 16
            next_trading_day = get_next_trading_day(headline_day)
            next_day_prices = stock_prices[stock_prices['Date'].dt.date == next_trading_day]
            
            if len(next_day_prices) > 0:
                return {
                    'entry_price': next_day_prices.iloc[0]['Open'],
                    'exit_price': next_day_prices.iloc[0]['Close']
                }
        
        return {
            'entry_price': None,
            'exit_price': None
        }
    
    print("Processing headlines...")
    # Apply the price lookup to each headline
    prices_data = result_df.apply(get_trading_prices, axis=1)
    
    # Add the new columns
    result_df['entry_price'] = prices_data.apply(lambda x: x['entry_price'])
    result_df['exit_price'] = prices_data.apply(lambda x: x['exit_price'])
    
    # Calculate returns
    result_df['return_pct'] = (
        (result_df['exit_price'] - result_df['entry_price']) / 
        result_df['entry_price'] * 100
    ).round(2)
    print("Finished!")
    
    return result_df

input_path = 'output2/filtered_data8.csv'
output_path = 'output2/filtered_data9.csv'

df = pd.read_csv(input_path)
filtered_df = filter_headlines_by_trading_session(df)
result_df = get_trading_returns(filtered_df, batch_size=50)  
valid_trades_df = result_df.dropna(subset=['return_pct'])
valid_trades_df.to_csv(output_path, index=False)

21 stocks not found in cache, attempting to fetch...


  0%|          | 0/1 [00:00<?, ?it/s]

$ARRY: possibly delisted; no price data found  (1d 2018-10-01 -> 2020-01-05) (Yahoo error = "Data doesn't exist for startDate = 1538366400, endDate = 1578200400")
$BAM: possibly delisted; no price data found  (1d 2018-10-01 -> 2020-01-05) (Yahoo error = "Data doesn't exist for startDate = 1538366400, endDate = 1578200400")
$BEAT: possibly delisted; no price data found  (1d 2018-10-01 -> 2020-01-05) (Yahoo error = "Data doesn't exist for startDate = 1538366400, endDate = 1578200400")
$BTX: possibly delisted; no price data found  (1d 2018-10-01 -> 2020-01-05) (Yahoo error = "Data doesn't exist for startDate = 1538366400, endDate = 1578200400")
$CR: possibly delisted; no price data found  (1d 2018-10-01 -> 2020-01-05) (Yahoo error = "Data doesn't exist for startDate = 1538366400, endDate = 1578200400")
$CRC: possibly delisted; no price data found  (1d 2018-10-01 -> 2020-01-05) (Yahoo error = "Data doesn't exist for startDate = 1538366400, endDate = 1578200400")
$CUB: possibly delisted; no

Processing headlines...
Finished!


## Stratified Sampling
Take a representative sample of the dataset using stratified sampling to maintain seasonal distribution, while reducing the data size for LLM processing.

Performing sampling early in the pipeline would have greatly sped up some of the above operations. However, it was done last so that different samples could easily be drawn from the dataset.

In [7]:
def stratified_stock_sample(df, fraction):
    df['date'] = pd.to_datetime(df['date'])
    
    # Create month-year column for stratification
    df['month_year'] = df['date'].dt.to_period('M')
    
    # Get all unique stocks and their corresponding month-years
    stock_info = df[['stock', 'month_year']].drop_duplicates()
    
    # Group by stock to get unique stocks
    unique_stocks = df['stock'].unique()
    print(f"Total unique stocks in dataset: {len(unique_stocks)}")
    
    # Group the unique stock-month combinations by month-year
    grouped_by_month = stock_info.groupby('month_year')
    
    # Initialize an empty list to store sampled stock-month combinations
    sampled_stock_months = []
    
    # For each month-year, sample a proportional number of stocks
    for month, group in grouped_by_month:
        # Calculate how many stocks to sample from this month
        n_to_sample = max(1, int(len(group) * fraction))
        
        # Sample stocks from this month
        month_sample = group.sample(n=n_to_sample, random_state=42)
        
        # Add to our list of sampled stock-month combinations
        sampled_stock_months.append(month_sample)
    
    # Combine all the sampled stock-month combinations
    sampled_stock_months_df = pd.concat(sampled_stock_months)
    
    print(f"Total sampled stock-month combinations: {len(sampled_stock_months_df)}")
    
    # Return the dataframe rows corresponding to the sampled stock-month combinations
    sampled_df = df.merge(sampled_stock_months_df, on=['stock', 'month_year'])
    
    return sampled_df

# Sampling rate
sample_fraction = 0.05
input_path = 'output2/filtered_data9.csv'
output_path = 'output2/filtered_data10.csv'
df = pd.read_csv(input_path)
sampled_df = stratified_stock_sample(df, sample_fraction)

# Show the results
print(f"Original headlines: {len(df)}")
print(f"Sampled headlines: {len(sampled_df)}")

print(f"Original trading days: {df['date'].nunique()}")
print(f"Sampled trading days: {sampled_df['date'].nunique()}")


# Verify the month distribution in original vs sampled data
original_monthly_dist = df.groupby(pd.to_datetime(df['date']).dt.to_period('M')).size()
sampled_monthly_dist = sampled_df.groupby(sampled_df['date'].dt.to_period('M')).size()

# Compare distributions
print("\nOriginal monthly distribution:")
print(original_monthly_dist)
print("\nSampled monthly distribution:")
print(sampled_monthly_dist)

# Verify stock distribution
original_stock_dist = df.groupby('stock').size().sort_values(ascending=False).head(10)
sampled_stock_dist = sampled_df.groupby('stock').size().sort_values(ascending=False).head(10)

print("\nTop 10 stocks in original data:")
print(original_stock_dist)
print("\nTop 10 stocks in sampled data:")
print(sampled_stock_dist)

# Save results
sampled_df.to_csv(output_path, index=False)

Total unique stocks in dataset: 2090
Total sampled stock-month combinations: 895
Original headlines: 59257
Sampled headlines: 2995
Original trading days: 324
Sampled trading days: 312

Original monthly distribution:
date
2018-10    4621
2018-11    3713
2018-12    2541
2019-01    3694
2019-02    3940
2019-03    3008
2019-04    3916
2019-05    4532
2019-06    2842
2019-07    4650
2019-08    4257
2019-09    3879
2019-10    5737
2019-11    4493
2019-12    3434
Freq: M, dtype: int64

Sampled monthly distribution:
date
2018-10    182
2018-11    174
2018-12    120
2019-01    263
2019-02    179
2019-03    158
2019-04    154
2019-05    266
2019-06    160
2019-07    214
2019-08    205
2019-09    210
2019-10    338
2019-11    213
2019-12    159
Freq: M, dtype: int64

Top 10 stocks in original data:
stock
GOOGL    500
NFLX     482
TSLA     468
GOOG     366
PCG      328
BABA     310
MU       291
QCOM     266
NVDA     261
MRK      259
dtype: int64

Top 10 stocks in sampled data:
stock
PCG      77
NF